# Comparing configurations with stackelberg players with and without observabilities

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Mattia\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
import sys
sys.path.append('../')

In [3]:
import source.players as pl
import source.parsers as parsers
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.runner as runner
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.util as util

import concurrent.futures
import pkgutil
import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
import os
import re
from copy import deepcopy, copy
import datetime
from contextlib import redirect_stdout

In [4]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [5]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [6]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)

<module 'source.util' from '..\\source\\util.py'>

In [7]:
time_horizon = 1000
T = 5
#targets = util.gen_targets(T, low=1, high=10)
targets = [5, 3, 8, 4, 9]
values = tuple((v, v) for v in targets)
#obs = gen_observabilities_correlated_with_values(values)
obs = {0: 0.9, 1: 0.9, 2: 0.75, 3: 0.9, 4: 0.9}
obs_vect = [obs.get(i) for i in range(len(targets))]
g = game.Game(values, time_horizon)
g.attackers = [1]
g.defenders = [0]
a = [attackers.StackelbergAttacker(g, 1)]
d = [base_defenders.StackelbergDefender(g, 0)]
p = [attackers.ObservingStackelbergAttacker(g, 1, 1), attackers.StackelbergAttacker(g, 1)]
g.set_players(d, a, p)

go1 = game.GameWithObservabilities(values, time_horizon)
go1.attackers = [1]
go1.defenders = [0]
a1 = [attackers.StackelbergAttacker(go1, 1)]
d1 = [base_defenders.StackelbergDefender(go1, 0)]
p1 = [attackers.ObservingStackelbergAttacker(go1, 1), attackers.StackelbergAttacker(go1, 1)]
go1.set_players(d1, a1, p1)

go2 = game.GameWithObservabilities(values, time_horizon)
go2.attackers = [1]
go2.defenders = [0]
a2 = [attackers.ObservingStackelbergAttacker(go2, 1)]
d2 = [base_defenders.StackelbergDefender(go2, 0, 1)]
p2 = [attackers.ObservingStackelbergAttacker(go2, 1), attackers.StackelbergAttacker(go2, 1)]
go2.set_players(d2, a2, p2)

go3 = game.GameWithObservabilities(values, time_horizon)
go3.attackers = [1]
go3.defenders = [0]
a3 = [attackers.ObservingStackelbergAttacker(go3, 1)]
d3 = [base_defenders.ObservingStackelbergDefender(go3, 0)]
p3 = [attackers.ObservingStackelbergAttacker(go3, 1), attackers.StackelbergAttacker(go3, 1)]
go3.set_players(d3, a3, p3)

In [8]:
def print_header(targets, profiles, observabilities):
    return "Name,T," + ",".join(str(i) for i in range(len(targets))) + ",Defender,Attacker," + ",".join(["Profile," for x in range(len(profiles[0]['others']))]) + ",".join(["Obs" + str(i) for i in range(len(observabilities))]) + "\n" 

In [9]:
def print_row(targets, game_type, time_horizon,d,p,obs):
    return ",".join([str(i) for i in ([d+"_vs_"+p["attacker"]+"_"+game_type] + [time_horizon] + targets + [d] + [p["attacker"]] + [x for x in p["others"]] + obs) ])+"\n"

In [10]:
profiles = []
profiles.append({"attacker": util.print_adv(a[0]), 
                 "others": [util.print_adv(p[0])]
                })
profiles1 = []
profiles1.append({"attacker": util.print_adv(a2[0]), 
                 "others": [util.print_adv(p2[1])]
                 })

In [11]:
(print_header(targets, profiles, obs_vect))

'Name,T,0,1,2,3,4,Defender,Attacker,Profile,Obs0,Obs1,Obs2,Obs3,Obs4\n'

In [12]:
defender =["sta_def", "sta_def","sta_def", "obsta_def"]

In [13]:
folder = "../Experiments/Experiment2"
batch_name = "game_no_obs_vs_obs"
batch_file = batch_name + ".csv"
batch_path = folder + "/" + batch_file

In [14]:
with open(batch_path, "w+") as f:
    f.write(print_header(targets, profiles, obs_vect))
    f.write(print_row(targets, "no_obs", time_horizon,defender[0],profiles[0],[]))
    f.write(print_row(targets, "obs", time_horizon,defender[1],profiles[0], obs_vect))
    f.write(print_row(targets, "obs", time_horizon,defender[2],profiles1[0], obs_vect))
    f.write(print_row(targets, "obs", time_horizon,defender[3],profiles1[0], obs_vect))

In [15]:
pd.read_csv(batch_path)

,Name,T,0,1,2,3,4,Defender,Attacker,Profile,Obs0,Obs1,Obs2,Obs3,Obs4
0,sta_def_vs_sta1_no_obs,1000,5,3,8,4,9,sta_def,sta1,sta1,NaN,NaN,NaN,NaN,NaN
1,sta_def_vs_sta1_obs,1000,5,3,8,4,9,sta_def,sta1,sta1,0.9,0.9,0.75,0.9,0.9
2,sta_def_vs_obsta1_obs,1000,5,3,8,4,9,sta_def,obsta1,obsta1,0.9,0.9,0.75,0.9,0.9
3,obsta_def_vs_obsta1_obs,1000,5,3,8,4,9,obsta_def,obsta1,obsta1,0.9,0.9,0.75,0.9,0.9


In [16]:
b = runner.Batch(batch_path, folder)

In [17]:
b.parse_batch()

In [18]:
batches = []
batches.append(b)

In [19]:
with concurrent.futures.ProcessPoolExecutor(None) as executor:
    futures = {}
    logger.info(str(datetime.datetime.now()))
    for b in batches:
        b.run(futures, executor, n=2)

In [20]:
with open('progress', 'w') as f:
    with redirect_stdout(f):
        for b in batches:
            b.collect(futures)
            confs = b.configurations
            print(b.results_folder_path)
            dlist = [{"name": c.game.players[0].__class__.name,
                      "avgs": c.stats["exp_regret"], 
                      "lb": c.stats["lb_exp_regret"],
                      "ub": c.stats["ub_exp_regret"]}
                     for c in confs]
            util.plot_dicts(dlist, save=True, show=False, semilog=True,
                            path=b.results_folder_path,
                            title=confs[0].game.players[1].__class__.name)

C:\Users\Mattia\Anaconda3\lib\site-packages\matplotlib\ticker.py:2207: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  "Data has no positive values, and therefore cannot be "


RuntimeError: latex was not able to process the following string:
b'sta_def'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.19 (MiKTeX 2.9.6745 64-bit)
entering extended mode
(C:/Users/Mattia/.matplotlib/tex.cache/fa57695546d77ddf83a091f7bfbbd85b.tex
LaTeX2e <2018-04-01> patch level 5
("C:\Users\Mattia\AppData\Local\Programs\MiKTeX 2.9\tex\latex\base\article.cls"
Document Class: article 2014/09/29 v1.4h Standard LaTeX document class
("C:\Users\Mattia\AppData\Local\Programs\MiKTeX 2.9\tex\latex\base\size10.clo")
)
("C:\Users\Mattia\AppData\Local\Programs\MiKTeX 2.9\tex\latex\type1cm\type1cm.s
ty")
("C:\Users\Mattia\AppData\Local\Programs\MiKTeX 2.9\tex\latex\psnfss\helvet.sty
"
("C:\Users\Mattia\AppData\Local\Programs\MiKTeX 2.9\tex\latex\graphics\keyval.s
ty"))
("C:\Users\Mattia\AppData\Local\Programs\MiKTeX 2.9\tex\latex\base\textcomp.sty
" ("C:\Users\Mattia\AppData\Local\Programs\MiKTeX 2.9\tex\latex\base\ts1enc.def
" ("C:\Use

C:\Users\Mattia\Anaconda3\lib\site-packages\matplotlib\font_manager.py:1331: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


RuntimeError: latex was not able to process the following string:
b'sta_def'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.19 (MiKTeX 2.9.6745 64-bit)
entering extended mode
(C:/Users/Mattia/.matplotlib/tex.cache/fa57695546d77ddf83a091f7bfbbd85b.tex
LaTeX2e <2018-04-01> patch level 5
("C:\Users\Mattia\AppData\Local\Programs\MiKTeX 2.9\tex\latex\base\article.cls"
Document Class: article 2014/09/29 v1.4h Standard LaTeX document class
("C:\Users\Mattia\AppData\Local\Programs\MiKTeX 2.9\tex\latex\base\size10.clo")
)
("C:\Users\Mattia\AppData\Local\Programs\MiKTeX 2.9\tex\latex\type1cm\type1cm.s
ty")
("C:\Users\Mattia\AppData\Local\Programs\MiKTeX 2.9\tex\latex\psnfss\helvet.sty
"
("C:\Users\Mattia\AppData\Local\Programs\MiKTeX 2.9\tex\latex\graphics\keyval.s
ty"))
("C:\Users\Mattia\AppData\Local\Programs\MiKTeX 2.9\tex\latex\base\textcomp.sty
" ("C:\Users\Mattia\AppData\Local\Programs\MiKTeX 2.9\tex\latex\base\ts1enc.def
" ("C:\Use

<Figure size 432x288 with 1 Axes>

In [21]:
gm = deepcopy(batches[0].configurations[2].game)

In [22]:
print(gm)

1000,5.0,3.0,8.0,4.0,9.0,obsta1,sta_def1,obsta1


In [23]:
isinstance(gm, game.GameWithObservabilities)

True

In [24]:
gm.players

{1: <ObservingStackelbergAttacker id:1 resources:1>,
 0: <StackelbergDefender id:0 resources:1>}

In [25]:
gm.observabilities

{0: 0.9, 1: 0.9, 2: 0.75, 3: 0.9, 4: 0.9}

In [26]:
e = runner.Experiment(gm)

In [27]:
for i in range(900):
    e.run_interaction_with_observations()
    print(e.game.strategy_history[-1])
    #print(util.game_str(e.game, lenght=7))
    print(e.game.history[-1])
    #print(list(e.agent.belief.pr.items())[2])

Academic license - for non-commercial use only
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [0]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.

{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1:

{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [0]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1:

{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [0]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1:

{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [0]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1:

{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1:

{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1:

{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [2]}
{0: [0.08280254777070063, 0.0, 0.4267515923566879, 0.0, 0.49044585987261147], 1: